## Data Loading

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [5]:
joined_df = pd.read_csv(r'../data/joined_df.csv')

#### Transformations Needed
- `Convert runtime to integer.`
- `Convert total sales to float`
- `Drop other missing values`
- `Convert release year to date (year)- left as string limited to 4 letters.`
- `Dropping budget column?`


## Summarized Transformations

In [9]:
def transform_df(data) -> pd.DataFrame:
    """
    Transform joined dataframe 
    Returns:
        _type_: _description_
    """
    joined_df = data
    joined_df = joined_df.dropna(
        subset=["votes_count", "rating", "description", "release_year"]
    )
    joined_df["runtime"] = joined_df["runtime"].astype(str).str.extract(r"(\d+)")
    joined_df["runtime"] = pd.to_numeric(joined_df["runtime"], errors="coerce")
    joined_df["total_sales"] = (
        joined_df["total_sales"].astype(str).replace(r"\,", "", regex=True)
    )
    joined_df["total_sales"] = pd.to_numeric(joined_df["total_sales"], errors="coerce")
    joined_df.loc[joined_df["release_year"] == "PG", "release_year"] = "1995"
    joined_df["release_year"] = joined_df["release_year"].astype(int).astype(str)

    joined_df["rating"] = joined_df["rating"].astype(float)
    joined_df["votes_count"] = joined_df["votes_count"].astype(float)
    cols_to_drop = ["budget, metascore"]
    for col in cols_to_drop:
        if col in joined_df.columns.to_list():
            joined_df = joined_df.drop(columns=col)
    return joined_df

## Data Preview

In [14]:
ac = transform_df(joined_df)
ac = ac.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
ac.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1672 entries, 0 to 1691
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1672 non-null   object 
 1   release_year  1672 non-null   object 
 2   runtime       1359 non-null   float64
 3   genres        1666 non-null   object 
 4   rating        1672 non-null   float64
 5   description   1672 non-null   object 
 6   director      999 non-null    object 
 7   votes_count   1672 non-null   float64
 8   total_sales   1050 non-null   float64
 9   type          1672 non-null   object 
dtypes: float64(4), object(6)
memory usage: 143.7+ KB


C:\Users\APIN-PC\AppData\Local\Temp\ipykernel_27316\1518329374.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df["runtime"] = joined_df["runtime"].astype(str).str.extract(r"(\d+)")
C:\Users\APIN-PC\AppData\Local\Temp\ipykernel_27316\1518329374.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df["runtime"] = pd.to_numeric(joined_df["runtime"], errors="coerce")
C:\Users\APIN-PC\AppData\Local\Temp\ipykernel_27316\1518329374.py:13: SettingWithCopyWarning: 
A value is trying to be set on 

In [15]:
ac.head(2)

,title,release_year,runtime,genres,rating,description,director,votes_count,total_sales,type
0,The Shawshank Redemption,1994,142.0,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,2343110.0,28341469.0,movie
1,The Godfather,1972,175.0,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,1620367.0,134966411.0,movie


In [11]:
ac.release_year.unique()

array(['1994', '1972', '2008', '1974', '1957', '2003', '1993', '2010',
       '1999', '2001', '1966', '2002', '1990', '1980', '1975', '2020',
       '2019', '2014', '1998', '1997', '1995', '1991', '1977', '1962',
       '1954', '1946', '2011', '2006', '2000', '1988', '1985', '1968',
       '1960', '1942', '1936', '1931', '2018', '2017', '2016', '2012',
       '2009', '2007', '1984', '1981', '1979', '1971', '1963', '1964',
       '1950', '1940', '2013', '2005', '2004', '1992', '1987', '1986',
       '1983', '1976', '1973', '1965', '1959', '1958', '1952', '1948',
       '1944', '1941', '1927', '1921', '2015', '1996', '1989', '1978',
       '1961', '1955', '1953', '1925', '1924', '1982', '1967', '1951',
       '1949', '1939', '1937', '1934', '1928', '1926', '1920', '1970',
       '1969', '1956', '1947', '1945', '1930', '1938', '1935', '1933',
       '1932', '1922', '1943', '2023', '2024', '2022', '2021', '2025'],
      dtype=object)

In [20]:
joined_df[joined_df['type'] == 'tvSeries'] 

ac[ac['type'] == 'tvSeries'].head(2)

,title,release_year,runtime,genres,rating,description,director,votes_count,total_sales,type
1030,Breaking Bad,2008,NaN,"Crime, Drama, Thriller",9.5,A chemistry teacher diagnosed with inoperable ...,NaN,2361775.0,NaN,tvSeries
1035,The Wire,2002,NaN,"Crime, Drama, Thriller",9.3,"The Baltimore drug scene, as seen through the ...",NaN,407612.0,NaN,tvSeries


I intentionally left missing values so as to show some peculiarities of the scraped data.
E.g Type column consists of -> movie, tvseries and tvshortseries, 
Only movie have record for runtime. Both tvseries and tvshortseries are null. 
It could mean that tvseries are still running or that the IMDb APIs does not keep record of series runtime. 